In [2]:
# Import Packages

import requests
import pandas as pd
import csv
import os
from sodapy import Socrata



# Download Delay Data, untrasformed, from API Endpoints

In [24]:
#Wait Assessment 2020 Onwards - "https://data.ny.gov/resource/swky-c3v4.csv"

# Create a directory to store the csv files
os.makedirs("data", exist_ok=True)

# Download the csv file
wait_assessment_2020 = "https://data.ny.gov/resource/swky-c3v4.csv?$limit=1000000000"
response = requests.get(wait_assessment_2020)

# Save the csv file to disk
csv_file = "wait_assessment_2020.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
wait_assessment_2020 = pd.read_csv(csv_file)

In [22]:
# Column and row count
print(wait_assessment_2020.shape)

# Describe the dataframe
wait_assessment_2020.describe(include="all")

(54332, 11)


,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment,Month,Year
count,54332,54332,54332.000000,54332,54332,54332,5.433200e+04,5.433200e+04,54072.000000,54332,54332
unique,47,7,NaN,4,334,3,NaN,NaN,NaN,4,12
top,2022-01,Queens,NaN,LCL/LTD,BX1,Off-Peak,NaN,NaN,NaN,2022,01
freq,1279,17494,NaN,41572,188,27146,NaN,NaN,NaN,14043,4761
mean,NaN,NaN,1.444269,NaN,NaN,NaN,6.751509e+03,8.751136e+03,0.782217,NaN,NaN
std,NaN,NaN,0.496889,NaN,NaN,NaN,9.251481e+04,1.200115e+05,0.093447,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,NaN,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,8.030000e+02,1.038750e+03,0.720797,NaN,NaN
50%,NaN,NaN,1.000000,NaN,NaN,NaN,1.941000e+03,2.505000e+03,0.787508,NaN,NaN
75%,NaN,NaN,2.000000,NaN,NaN,NaN,4.383000e+03,5.841000e+03,0.850317,NaN,NaN


In [23]:
# Column name and data type
print(wait_assessment_2020.dtypes)

month                            object
borough                          object
day_type                          int64
trip_type                        object
route_id                         object
period                           object
number_of_trips_passing_wait      int64
number_of_scheduled_trips         int64
wait_assessment                 float64
Month                            object
Year                             object
dtype: object


In [25]:
# Split the month and year into separate columns
wait_assessment_2020[["Year", "Month"]] = wait_assessment_2020["month"].str.split("-", expand=True)

In [26]:
# Describe the dataframe
wait_assessment_2020.describe(include="all")

,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment,Year,Month
count,54332,54332,54332.000000,54332,54332,54332,5.433200e+04,5.433200e+04,54072.000000,54332,54332
unique,47,7,NaN,4,334,3,NaN,NaN,NaN,4,12
top,2022-01,Queens,NaN,LCL/LTD,BX1,Off-Peak,NaN,NaN,NaN,2022,01
freq,1279,17494,NaN,41572,188,27146,NaN,NaN,NaN,14043,4761
mean,NaN,NaN,1.444269,NaN,NaN,NaN,6.751509e+03,8.751136e+03,0.782217,NaN,NaN
std,NaN,NaN,0.496889,NaN,NaN,NaN,9.251481e+04,1.200115e+05,0.093447,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,NaN,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,8.030000e+02,1.038750e+03,0.720797,NaN,NaN
50%,NaN,NaN,1.000000,NaN,NaN,NaN,1.941000e+03,2.505000e+03,0.787508,NaN,NaN
75%,NaN,NaN,2.000000,NaN,NaN,NaN,4.383000e+03,5.841000e+03,0.850317,NaN,NaN


In [27]:
# Wait Assessment 2015 - 2019 - "https://data.ny.gov/resource/bmix-dpzc.csv"

# Download the csv file
wait_assessment_2015 = "https://data.ny.gov/resource/bmix-dpzc.csv?$limit=1000000000"
response = requests.get(wait_assessment_2015)

# Save the csv file to disk
csv_file = "wait_assessment_2015.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
wait_assessment_2015 = pd.read_csv(csv_file)

In [28]:
# Split the month and year into separate columns
wait_assessment_2015[["Year", "Month"]] = wait_assessment_2015["month"].str.split("-", expand=True)

In [29]:
# Merge the two dataframes
wait_assessment_2015_2020 = pd.concat([wait_assessment_2020, wait_assessment_2015], axis=0, ignore_index=True)

In [30]:
# PRint the shape of the dataframe
print(wait_assessment_2015_2020.shape)

# Describe the dataframe
wait_assessment_2015_2020.describe(include="all")


(123607, 11)


,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment,Year,Month
count,123607,123607,123607.000000,123607,123607,123607,1.236070e+05,1.236070e+05,123072.000000,123607,123607
unique,107,7,NaN,4,369,3,NaN,NaN,NaN,9,12
top,2022-01,Queens,NaN,LCL/LTD,BX1,Off-Peak,NaN,NaN,NaN,2022,01
freq,1279,40085,NaN,95584,428,61703,NaN,NaN,NaN,14043,11022
mean,NaN,NaN,1.447556,NaN,NaN,NaN,7.279524e+03,9.368812e+03,0.789280,NaN,NaN
std,NaN,NaN,0.497244,NaN,NaN,NaN,9.933579e+04,1.280339e+05,0.089842,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,NaN,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,8.590000e+02,1.097000e+03,0.730558,NaN,NaN
50%,NaN,NaN,1.000000,NaN,NaN,NaN,2.119000e+03,2.710000e+03,0.794101,NaN,NaN
75%,NaN,NaN,2.000000,NaN,NaN,NaN,4.731000e+03,6.260000e+03,0.853659,NaN,NaN


In [15]:
# Print the shape of the dataframe
print(wait_assessment_2015.shape)

# Describe the dataframe
wait_assessment_2015.describe(include="all")

(69275, 9)


,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment
count,69275,69275,69275.000000,69275,69275,69275,6.927500e+04,6.927500e+04,69000.000000
unique,60,6,NaN,4,358,3,NaN,NaN,NaN
top,2017-01,Queens,NaN,LCL/LTD,X27,Peak,NaN,NaN,NaN
freq,1271,22591,NaN,54012,244,34598,NaN,NaN,NaN
mean,NaN,NaN,1.450134,NaN,NaN,NaN,7.693643e+03,9.853252e+03,0.794816
std,NaN,NaN,0.497511,NaN,NaN,NaN,1.043728e+05,1.339891e+05,0.086511
min,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000
25%,NaN,NaN,1.000000,NaN,NaN,NaN,9.100000e+02,1.146000e+03,0.738466
50%,NaN,NaN,1.000000,NaN,NaN,NaN,2.250000e+03,2.872000e+03,0.798664
75%,NaN,NaN,2.000000,NaN,NaN,NaN,5.017500e+03,6.610000e+03,0.856130


In [22]:
# Service Deliviered 2020 Onwards - "https://data.ny.gov/resource/2e6s-9gpm.csv"

# Download the csv file
service_delivered_2020 = "https://data.ny.gov/resource/2e6s-9gpm.csv?$limit=1000000000"
response = requests.get(service_delivered_2020)

# Save the csv file to disk
csv_file = "service_delivered_2020.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
service_delivered_2020 = pd.read_csv(csv_file)

In [24]:
# Service Deliviered 2015 - 2019 - "https://data.ny.gov/resource/tw28-zvtk.csv"

# Download the csv file
service_delivered_2015 = "https://data.ny.gov/resource/tw28-zvtk.csv?$limit=1000000000"
response = requests.get(service_delivered_2015)

# Save the csv file to disk
csv_file = "service_delivered_2015.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
service_delivered_2015 = pd.read_csv(csv_file)

# Download Traffic Data, untransformed, from API Endpoints

In [32]:
# Hourly Traffic on (MTA) Bridges and Tunnels: Beginning 2010 - "https://data.ny.gov/resource/qzve-kjga.csv"

# Download the csv file
hourly_traffic = "https://data.ny.gov/resource/qzve-kjga.csv?$limit=1000000000"
response = requests.get(hourly_traffic)

# Save the csv file to disk
csv_file = "hourly_traffic.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
hourly_traffic = pd.read_csv(csv_file)


In [43]:
# Print the shape of the dataframe
print(hourly_traffic.shape)

# Describe the dataframe
hourly_traffic.describe(include="all")

(89935, 9)


,date,plaza_id,direction,hour,vehicles_e_zpass,vehicles_vtoll,total_vehicles,Year,Month
count,89935,89935.000000,89935,89935.000000,89935.000000,89935.000000,89935.000000,89935,89935
unique,5029,NaN,2,NaN,NaN,NaN,NaN,15,12
top,2010-01-01T00:00:00.000,NaN,I,NaN,NaN,NaN,NaN,2010,01
freq,19,NaN,45726,NaN,NaN,NaN,NaN,6935,8039
mean,NaN,14.217513,NaN,23.993429,23.993429,23.993429,47.986857,NaN,NaN
std,NaN,10.536804,NaN,0.141674,0.141674,0.141674,0.283348,NaN,NaN
min,NaN,1.000000,NaN,12.000000,12.000000,12.000000,24.000000,NaN,NaN
25%,NaN,5.000000,NaN,24.000000,24.000000,24.000000,48.000000,NaN,NaN
50%,NaN,9.000000,NaN,24.000000,24.000000,24.000000,48.000000,NaN,NaN
75%,NaN,25.000000,NaN,24.000000,24.000000,24.000000,48.000000,NaN,NaN


In [ ]:
# Create a new column named total_vehicles by adding the vehicles_e_zpass and vehicles_vtoll columns
hourly_traffic["total_vehicles"] = hourly_traffic["vehicles_e_zpass"] + hourly_traffic["vehicles_vtoll"]

In [40]:
# Extract the year from the date_time column
hourly_traffic["Year"] = hourly_traffic["date"].str[:4]

# Extract the month from the date column
hourly_traffic["Month"] = hourly_traffic["date"].str[5:7]

In [44]:
# Transform the dataframe to group by the Year, Month, plaza_id, and direction columns, giving the sum of the total_vehicles, vehicles_e_zpass, and vehicles_vtoll columns
hourly_traffic_transformed = hourly_traffic.groupby(["Year", "Month", "plaza_id", "direction"]).agg({"total_vehicles": "sum", "vehicles_e_zpass": "sum", "vehicles_vtoll": "sum"}).reset_index()    

In [ ]:
# Print the shape of the dataframe
print(hourly_traffic_transformed.shape)

# Describe the dataframe
hourly_traffic_transformed.describe(include="all")


# The following csv is a large file available at https://data.cityofnewyork.us/resource/7ym2-wayt
# The full file has been downloaded and is used in the following script.
# The aggregate output will also be made available and it is recommended this is the version to work with

In [3]:
# Open a CSV file and write it to a df using pandas
# Open the CSV file

traffic_speeds = pd.read_csv('DOT_Traffic_Speeds_NBE_20240121.csv')

C:\Users\Paul\AppData\Local\Temp\ipykernel_14724\3722983806.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_speeds = pd.read_csv('DOT_Traffic_Speeds_NBE_20240121.csv')


In [4]:
ts_backup = traffic_speeds

In [5]:
# Print the shape of the dataframe
print(traffic_speeds.shape)

# Describe the dataframe
traffic_speeds.describe(include="all")

(11354347, 13)


,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
count,11354347.0,1.135434e+07,1.135434e+07,1.135434e+07,11354344,1.135434e+07,11354344,11354343,11354343,11354343,1.135434e+07,11354343,11354343
unique,281.0,NaN,NaN,NaN,662468,NaN,138,1171,42,6,NaN,6,139
top,217.0,NaN,NaN,NaN,09/20/2019 07:27:03 AM,NaN,"40.63092,-74.14592 40.62975,-74.14593 40.62877...",w~mwFjisbMc@}HkCe^gBoZe@iQe@mJSaBYmA_A{BuBqCcC...,BBBBBBBBBBBBBBB,NYC_DOT_LIC,NaN,Queens,FDR N Catherine Slip - 25th St
freq,83674.0,NaN,NaN,NaN,101,NaN,167568,83794,831857,8872375,NaN,3305143,83794
mean,NaN,3.493317e+01,2.213807e+02,-1.333680e+01,NaN,4.575233e+06,NaN,NaN,NaN,NaN,4.575233e+06,NaN,NaN
std,NaN,1.923900e+01,1.599458e+03,3.419279e+01,NaN,1.026174e+05,NaN,NaN,NaN,NaN,1.026174e+05,NaN,NaN
min,NaN,0.000000e+00,-5.700000e+01,-1.010000e+02,NaN,4.329472e+06,NaN,NaN,NaN,NaN,4.329472e+06,NaN,NaN
25%,NaN,1.864000e+01,7.200000e+01,0.000000e+00,NaN,4.616192e+06,NaN,NaN,NaN,NaN,4.616192e+06,NaN,NaN
50%,NaN,4.101000e+01,1.350000e+02,0.000000e+00,NaN,4.616235e+06,NaN,NaN,NaN,NaN,4.616235e+06,NaN,NaN
75%,NaN,5.095000e+01,2.350000e+02,0.000000e+00,NaN,4.616332e+06,NaN,NaN,NaN,NaN,4.616332e+06,NaN,NaN


In [6]:
# Keep the Speed, travel_time, status, data_as_of, link_id, and borough columns
traffic_speeds = traffic_speeds[['SPEED', 'TRAVEL_TIME', 'STATUS', 'DATA_AS_OF', 'LINK_ID', 'BOROUGH']]

In [7]:
# Extract the year from the data_as_of column which is in the format of 09/20/2019 07:27:03 AM
traffic_speeds["Year"] = traffic_speeds["DATA_AS_OF"].str[6:10]

# Extract the month from the data_as_of column which is in the format of 09/20/2019 07:27:03 AM
traffic_speeds["Month"] = traffic_speeds["DATA_AS_OF"].str[0:2]

C:\Users\Paul\AppData\Local\Temp\ipykernel_14724\2582025277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_speeds["Year"] = traffic_speeds["DATA_AS_OF"].str[6:10]
C:\Users\Paul\AppData\Local\Temp\ipykernel_14724\2582025277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_speeds["Month"] = traffic_speeds["DATA_AS_OF"].str[0:2]


In [11]:
# Group the dataframe by the Year, Month, and BOROUGH and LINK_ID, columns, giving the mean of the speed and travel_time columns
traffic_speeds_transformed = traffic_speeds.groupby(["Year", "Month", "BOROUGH", "LINK_ID"]).agg({"SPEED": "mean", "TRAVEL_TIME": "mean"}).reset_index()

In [12]:
# Print the shape of the dataframe
print(traffic_speeds_transformed.shape)

# Describe the dataframe
traffic_speeds_transformed.describe(include="all")

(2018, 6)


,Year,Month,BOROUGH,LINK_ID,SPEED,TRAVEL_TIME
count,2018,2018,2018,2.018000e+03,2018.000000,2018.000000
unique,3,12,6,NaN,NaN,NaN
top,2019,03,Queens,NaN,NaN,NaN
freq,1497,274,572,NaN,NaN,NaN
mean,NaN,NaN,NaN,4.574448e+06,35.865531,210.040639
std,NaN,NaN,NaN,1.045490e+05,15.429251,185.288915
min,NaN,NaN,NaN,4.329472e+06,0.000000,0.000000
25%,NaN,NaN,NaN,4.616192e+06,26.920289,91.850000
50%,NaN,NaN,NaN,4.616235e+06,39.669432,157.234479
75%,NaN,NaN,NaN,4.616329e+06,47.172769,270.629374


In [13]:
# save the dataframe to a csv file
traffic_speeds_transformed.to_csv("traffic_speeds_transformed.csv", index=False)

# Take traffic_speeds_transformed.csv

In [40]:
# Convert to pandas DataFrame
automated_traffic_volume_counts = pd.DataFrame.from_records(results)

# Save the csv file to disk
csv_file = "automated_traffic_volume_counts.csv"
automated_traffic_volume_counts.to_csv(csv_file, index=False)


In [15]:
# https://data.cityofnewyork.us/Transportation/Automated-Traffic-Volume-Counts/7ym2-wayt

# Download the csv file
automated_traffic_volume_counts = "https://data.ny.gov/resource/7ym2-wayt.csv?$limit=1000000000"
response = requests.get(automated_traffic_volume_counts)

# Save the csv file to disk
csv_file = "automated_traffic_volume_counts.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

#Save the csv file to a dataframe    
automated_traffic_volume_counts = pd.read_csv(csv_file)

In [16]:
# Use a select and group by statement to get the count of the volume column grouped by the year, month, and roadway_name columns
automated_traffic_volume_counts_transformed = "https://data.ny.gov/resource/7ym2-wayt.csv?$select=Yr,M,Boro,count(RequestID)&$group==Yr,M,Boro&$limit=1000000000" 
response = requests.get(automated_traffic_volume_counts)

In [17]:
# Save the csv file to disk
csv_file = "automated_traffic_volume_counts1.csv"
with open(csv_file, "w") as f:
    f.write(response.text)

MemoryError: 

In [ ]:
# Save the csv file to a dataframe    
automated_traffic_volume_counts = pd.read_csv(csv_file)

In [41]:
automated_traffic_volume_counts.describe(include='all')

,boro,street,yr,m,sum_vol,count_requestid
count,17675,17675,17675,17675,17675,17675
unique,5,6750,18,12,16649,3417
top,Brooklyn,BROADWAY,2017,10,0,1344
freq,5735,127,1939,3147,14,837
